In [1]:
import tensorflow as tf

In [2]:
cluster_spec = tf.train.ClusterSpec({
    "ps": [
        "127.0.0.1:2221",  # /job:ps/task:0
    ],
    "worker": [
        "127.0.0.1:2223",  # /job:worker/task:0
        "127.0.0.1:2224",  # /job:worker/task:1
    ]})

In [3]:
server_ps0 = tf.train.Server(cluster_spec, job_name="ps", task_index=0)
server_worker0 = tf.train.Server(cluster_spec, job_name="worker", task_index=0)
server_worker1 = tf.train.Server(cluster_spec, job_name="worker", task_index=1)

In [4]:
with tf.device('/job:ps/task:0'):
    a = tf.Variable(1.0, name='a')

with tf.device('/job:worker/task:0'):
    tf.assign(a, a+1)
    
with tf.device('/job:worker/task:1'):
    tf.assign(a, a+1)

In [5]:
with tf.Session(server_worker0.target) as sess: #using the server
    sess.run(a.initializer)
    print(a.eval())

1.0


In [7]:
with tf.Session("grpc://127.0.0.1:2223") as sess: #using the server address
    print(a.eval())

1.0
